Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = '/home/smart/Documents/Deepankar/ML/tensorflow_tutorial/notMNIST/notMNIST_large/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.228564
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.351265
Minibatch accuracy: 68.8%
Validation accuracy: 50.2%
Minibatch loss at step 100: 1.543641
Minibatch accuracy: 56.2%
Validation accuracy: 59.9%
Minibatch loss at step 150: 0.734484
Minibatch accuracy: 75.0%
Validation accuracy: 76.5%
Minibatch loss at step 200: 0.980121
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 250: 1.066355
Minibatch accuracy: 68.8%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.489901
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 350: 0.724005
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.829488
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 450: 0.977899
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.686502
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.682827
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.805455
Minibatch accuracy: 25.0%
Validation accuracy: 41.4%
Minibatch loss at step 100: 1.505140
Minibatch accuracy: 50.0%
Validation accuracy: 50.2%
Minibatch loss at step 150: 0.566264
Minibatch accuracy: 81.2%
Validation accuracy: 71.7%
Minibatch loss at step 200: 0.843380
Minibatch accuracy: 68.8%
Validation accuracy: 71.7%
Minibatch loss at step 250: 1.058500
Minibatch accuracy: 62.5%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.574974
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.835173
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 400: 0.373736
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.832190
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.759978
Minibatch accuracy: 81.2%
Validation accuracy: 82.0%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [19]:
from IPython.core.debugger import Tracer

# define dimensions
batch_size = 128
image_size = 28
filter_size = 3
channels = 1
num_filters = 16

# define hyper-parameters
hidden_layers = 2
hidden_nodes = 1024
reg = 1e-2

# define variables
conv_params = {}
fc_params = {}
l2_pen = 0

# define model functions
def nn_fcnet(x, w, b):
    z = tf.matmul(x, w) + b
    return z

def nn_conv(x, w, b, strides):
    z = tf.nn.conv2d(x, w, strides=strides, padding='SAME') + b
    return z
    
def nn_relu(z):
    a = tf.nn.relu(z)
    return a

"""def nn_computeforward(a):
    # computes forward pass
    for h in xrange(hidden_layers):
        z = nn_feedforward(a, params['w%s' % (h+1)], params['b%s' %(h+1)])
        a = nn_relu(z)
    score = tf.matmul(a, params['w%s' % (hidden_layers+1)]) + params['b%s' % (hidden_layers+1)]
    
    return score"""

def model(x, conv_params, fc_params, strides=[1,1,1,1]):
    # convnet arch
    z1_conv = nn_conv(x, conv_params['w1'], conv_params['b1'], [1,1,1,1])
    a1_conv = nn_relu(z1_conv)
    z2_conv = nn_conv(a1_conv, conv_params['w2'], conv_params['b2'], strides)
    a2_conv = nn_relu(z2_conv)
    z3_conv = nn_conv(a2_conv, conv_params['w3'], conv_params['b3'], strides)
    a3_conv = nn_relu(z3_conv)
    # reshape output
    a3_shape = a3_conv.get_shape().as_list()
    a3_conv = tf.reshape(a3_conv, [a3_shape[0], a3_shape[1] * a3_shape[2] * a3_shape[3]])
    # fcnet arch
    z1 = nn_fcnet(a3_conv, fc_params['w1'], fc_params['b1'])
    a1 = nn_relu(z1)
    score = nn_fcnet(a1, fc_params['w2'], fc_params['b2'])
    
    return score

# instantiate new computational graph
new_graph = tf.Graph()
with new_graph.as_default():
    
    # declare placeholders for training data and define constants 
    # for validation and test data
    x_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size, image_size, channels))
    y_train = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels))
    x_val = tf.constant(valid_dataset)
    x_test = tf.constant(test_dataset)
    global_step = tf.Variable(0, trainable=False)
    initial_lr = 0.001
    
    # model
    # convnet weights
    conv_params['w1'] = tf.Variable(
        tf.truncated_normal([filter_size, filter_size, channels, num_filters], stddev= 1. / tf.sqrt(float(batch_size))))
    conv_params['b1'] = tf.Variable(tf.zeros([num_filters]))
    conv_params['w2'] = tf.Variable(
        tf.truncated_normal([filter_size, filter_size, num_filters, num_filters], stddev= 1. / tf.sqrt(float(batch_size))))
    conv_params['b2'] = tf.Variable(tf.zeros([num_filters]))
    conv_params['w3'] = tf.Variable(
        tf.truncated_normal([filter_size, filter_size, num_filters, num_filters], stddev= 1. / tf.sqrt(float(batch_size))))
    conv_params['b3'] = tf.Variable(tf.zeros([num_filters]))
    # fully-connected net weights
    fc_params['w1'] = tf.Variable(
        tf.truncated_normal([image_size // 4 * image_size // 4 * num_filters, hidden_nodes]))
    fc_params['b1'] = tf.Variable(tf.zeros(hidden_nodes))
    fc_params['w2'] = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    fc_params['b2'] = tf.Variable(tf.zeros(num_labels))
    
    
    score = model(x_train, conv_params, fc_params, [1,2,2,1])
    """# declare variables
    temp = np.repeat(hidden_nodes, hidden_layers)
    dims = [input_dim] + temp.tolist() + [num_labels]
    
    for h in xrange(hidden_layers+1):
        params['w%s' % (h+1)] = tf.Variable(tf.truncated_normal([dims[h], dims[h+1]]))
        params['b%s' %(h+1)] = tf.Variable(tf.zeros([dims[h+1]]))
    
    # compute forward pass
    score = nn_computeforward(x_train)
    
    # compute L2-penalty and cross-entropy loss
    for h in xrange(hidden_layers):
        l2_pen += tf.nn.l2_loss(params['w%s' % (h+1)])"""
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train, logits=score))
    lr = tf.train.exponential_decay(initial_lr, global_step, 10000, 0.1, staircase=True)
    
    # initialise the SGD optimiser
    optimiser = tf.train.AdamOptimizer(learning_rate=lr,beta1=0.7, beta2=0.999).minimize(loss, global_step=global_step)
    
    # predictions on each dataset
    y_train_pred = tf.nn.softmax(score)
    y_val_pred = tf.nn.softmax(model(x_val, conv_params, fc_params, [1,2,2,1]))
    y_test_pred = tf.nn.softmax(model(x_test, conv_params, fc_params, [1,2,2,1]))
    

In [20]:
num_steps = 40001

with tf.Session(graph=new_graph) as new_session:

    # randomly initialise global variables
    tf.global_variables_initializer().run()
    print("Initialised")
    
    for step in xrange(num_steps):
        # generate mini-batch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # initialise the feed_dict
        feed_dict = {x_train: batch_data, y_train: batch_labels}

        # run the session
        _, l, y_pred = new_session.run([optimiser, loss, y_train_pred], feed_dict=feed_dict)

        # print the predictions after every 500 steps
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(y_pred, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(y_val_pred.eval(), valid_labels))
            print("----####----")

    print("Test accuracy: %.1f%%" % accuracy(y_test_pred.eval(), test_labels))


Initialised
Minibatch loss at step 0: 43.192421
Minibatch accuracy: 10.9%
Validation accuracy: 10.2%
----####----
Minibatch loss at step 500: 0.722536
Minibatch accuracy: 82.0%
Validation accuracy: 82.4%
----####----
Minibatch loss at step 1000: 0.386536
Minibatch accuracy: 89.8%
Validation accuracy: 84.5%
----####----
Minibatch loss at step 1500: 0.375749
Minibatch accuracy: 87.5%
Validation accuracy: 86.1%
----####----
Minibatch loss at step 2000: 0.451267
Minibatch accuracy: 88.3%
Validation accuracy: 86.8%
----####----
Minibatch loss at step 2500: 0.448976
Minibatch accuracy: 86.7%
Validation accuracy: 87.2%
----####----
Minibatch loss at step 3000: 0.453058
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
----####----
Minibatch loss at step 3500: 0.448825
Minibatch accuracy: 85.9%
Validation accuracy: 88.2%
----####----
Minibatch loss at step 4000: 0.382694
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
----####----
Minibatch loss at step 4500: 0.537879
Minibatch accurac